In [1]:
import numpy as np
import re
import os
import pathlib
import tensorflow as tf
import tensorflow_text as text
from tensorflow import keras

2022-03-23 10:27:29.664568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-23 10:27:29.664644: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path_train = 'aclImdb_v1/train'


Reading files and labelling

In [3]:
data_arr, data_labels = [], []

for x in os.listdir(path_train):
    if not x == 'unsup':
        if x == 'pos':
            pos_path = os.path.join(path_train, x)
            for pos in os.listdir(pos_path):
                file_pos_path = os.path.join(pos_path, pos)
                with open(file_pos_path, 'r') as p:
                    data = p.readlines()
                    lines = []
                    for line in data:
                        s = line.replace('<br />', '')
                        s = re.sub('[.][.][.]*', '', s)
                        s = re.sub("([.,?!()])", r' \1', s)
                        s = s.split()
                        s = s[:60]
                        s = " ".join(s)
                        lines.append(s)
                        data_arr.append(lines)
                        lines= []
                        label = 1
                        data_labels.append(label)

        if x == 'neg':
            neg_path = os.path.join(path_train, x)
            for neg in os.listdir(neg_path):
                file_neg_path = os.path.join(neg_path, neg)
                with open(file_neg_path, 'r') as p:
                    data = p.readlines()
                    lines = []
                    for line in data:
                        s = line.replace('<br />', '')
                        s = re.sub('[.][.][.]*', '', s)
                        s = re.sub("([.,?!()])", r' \1', s)
                        s = s.split()
                        s = s[:60]
                        s = " ".join(s)
                        lines.append(s)
                        data_arr.append(lines)
                        lines = []
                        label = 0
                        data_labels.append(label)




In [5]:
path = "strings_tokenizer"
reload_bert = tf.saved_model.load(path)

In [45]:
tokens = reload_bert.tokenize(data_arr[0])

In [7]:
X_train = []
for x in data_arr:
    tokens = reload_bert.tokenize(x)
    token = tokens.numpy()
    X_train.append(token[0])


In [16]:
def to_dataset(data, labels):
    # data = tf.constant(data)
    ds = tf.data.Dataset.from_tensors((data, labels))
    ds = ds.batch(32)
    return ds



In [13]:
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post')

In [14]:
X_train_padded.shape

(102, 176)

In [17]:
dataset = to_dataset(X_train_padded, data_labels)

Token Embedding layer

In [132]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = x.shape[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
        

In [133]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [115]:
vocab_size = reload_bert.get_vocab_size()

In [116]:
maxlen = X_train_padded.shape[0]
maxlen

102

In [117]:
maxlens = X_train_padded.shape[1]
maxlens

176

In [118]:
embed_dim = 32
num_heads = 2
ff_dim = 32


In [134]:
inputs = keras.layers.Input(shape=(maxlens,))
embedding_layer = TokenAndPositionEmbedding(maxlens, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer = TransformerBlock(embed_dim, num_heads,ff_dim)
x = transformer(x)
x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(16, activation='relu')(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(2, activation='softmax')(x)

In [135]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [136]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [137]:
history = model.fit(np.asarray(X_train_padded), np.asarray(data_labels), epochs=20)

Epoch 1/20
4/4 [==============================] - 3s 106ms/step - loss: 0.7178 - accuracy: 0.4510
Epoch 2/20
4/4 [==============================] - 0s 103ms/step - loss: 0.6960 - accuracy: 0.5000
Epoch 3/20
4/4 [==============================] - 0s 97ms/step - loss: 0.6902 - accuracy: 0.5000
Epoch 4/20
4/4 [==============================] - 0s 105ms/step - loss: 0.6928 - accuracy: 0.5098
Epoch 5/20
4/4 [==============================] - 0s 98ms/step - loss: 0.6912 - accuracy: 0.5392
Epoch 6/20
4/4 [==============================] - 0s 98ms/step - loss: 0.6883 - accuracy: 0.5098
Epoch 7/20
4/4 [==============================] - 0s 100ms/step - loss: 0.6814 - accuracy: 0.6471
Epoch 8/20
4/4 [==============================] - 0s 98ms/step - loss: 0.6829 - accuracy: 0.6078
Epoch 9/20
4/4 [==============================] - 0s 97ms/step - loss: 0.6832 - accuracy: 0.5784
Epoch 10/20
4/4 [==============================] - 0s 99ms/step - loss: 0.6775 - accuracy: 0.5882
Epoch 11/20
4/4 [========

In [138]:
pred = np.asarray(X_train_padded)
pred

array([[  2, 105,  87, ...,   0,   0,   0],
       [  2,  50, 128, ...,   0,   0,   0],
       [  2,  39, 153, ...,   0,   0,   0],
       ...,
       [  2,  83, 207, ...,   0,   0,   0],
       [  2,  66,  73, ...,   0,   0,   0],
       [  2,  39,   9, ...,   0,   0,   0]], dtype=int32)

In [139]:
predi = model.evaluate(pred)

4/4 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [154]:
predii = model.predict(pred)

In [162]:
predictions = []
for x in predii:
    predictions.append(tf.argmax(x).numpy())

predictions


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

102